In [5]:
import numpy as np 
import os
# import skimage.io as io
# import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
# import skimage.io as io
# import skimage.transform as trans
import cv2
import numpy as np
import shutil
# from sklearn.metrics import jaccard_score
import matplotlib.pylab as plt

2023-02-12 18:19:13.927214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
dataset = "Hyper-Kvasir classification/unet_data"
train_path = dataset + "/Train"
val_path = dataset + "/Val"
test_path = dataset + "/Test"


train_images_path = train_path + "/images"
train_masks_path = train_path + "/masks"
val_images_path = val_path + "/images"
val_masks_path = val_path + "/masks"
test_images_path = test_path + "/images"
test_masks_path = test_path + "/masks"

len(os.listdir((train_images_path))), len(os.listdir((train_masks_path))), len(os.listdir((val_images_path))), len(os.listdir((val_masks_path))), len(os.listdir((test_images_path))), len(os.listdir((test_masks_path)))

(800, 800, 100, 100, 100, 100)

In [ ]:
# shutil.rmtree(train_images_path), shutil.rmtree(train_masks_path)
# shutil.rmtree(val_images_path), shutil.rmtree(val_masks_path)
# shutil.rmtree(test_images_path), shutil.rmtree(test_masks_path)

# os.mkdir(train_images_path), os.mkdir(train_masks_path)
# os.mkdir(val_images_path), os.mkdir(val_masks_path)
# os.mkdir(test_images_path), os.mkdir(test_masks_path)

In [ ]:
# for i in os.listdir('Hyper-Kvasir classification/unet_data/data/images'):
#     shutil.copy('Hyper-Kvasir classification/unet_data/data/images/'+i, 'Hyper-Kvasir classification/unet_data/Train/images')
#     shutil.copy('Hyper-Kvasir classification/unet_data/data/masks/'+i, 'Hyper-Kvasir classification/unet_data/Train/masks')
# for i in os.listdir('Hyper-Kvasir classification/unet_data/data/masks'):
#     shutil.copy('Hyper-Kvasir classification/unet_data/data/images/'+i, 'Hyper-Kvasir classification/unet_data/Train/images')
#     shutil.copy('Hyper-Kvasir classification/unet_data/data/masks/'+i, 'Hyper-Kvasir classification/unet_data/Train/masks')

In [ ]:
# for j in os.listdir('Hyper-Kvasir classification/unet_data/Train/images')[:100]:
#     shutil.move('Hyper-Kvasir classification/unet_data/Train/images/'+j, 'Hyper-Kvasir classification/unet_data/Val/images')
#     shutil.move('Hyper-Kvasir classification/unet_data/Train/masks/'+j, 'Hyper-Kvasir classification/unet_data/Val/masks')
# for l in os.listdir('Hyper-Kvasir classification/unet_data/Train/images')[:100]:
#     shutil.move('Hyper-Kvasir classification/unet_data/Train/images/'+l, 'Hyper-Kvasir classification/unet_data/Test/images')
#     shutil.move('Hyper-Kvasir classification/unet_data/Train/masks/'+l, 'Hyper-Kvasir classification/unet_data/Test/masks')

In [7]:

len(os.listdir((train_images_path))), len(os.listdir((train_masks_path))), len(os.listdir((val_images_path))), len(os.listdir((val_masks_path))), len(os.listdir((test_images_path))), len(os.listdir((test_masks_path)))

(800, 800, 100, 100, 100, 100)

In [ ]:
def train_data_generator():
    train_images = os.listdir(train_images_path)
    train_masks = os.listdir(train_masks_path)
    while True:
        for i in range(len(train_images)):
            img = cv2.imread(train_images_path + "/" + train_images[i])
            img = cv2.resize(img, (256, 256))
            img = img/255
            mask = cv2.imread(train_masks_path + "/" + train_images[i], 0)
            mask = cv2.resize(mask, (256, 256))
            mask = mask/255
            mask = np.expand_dims(mask, axis = 2)
            yield (img, mask)
def test_data_generator():
    val_images = os.listdir(val_images_path)
    val_masks = os.listdir(val_masks_path)
    while True:
        for i in range(len(val_images)):
            original_img = cv2.imread(val_images_path + "/" + val_images[i])
            img = cv2.resize(original_img, (256, 256))
            img = img/255
            original_mask = cv2.imread(val_masks_path + "/" + val_images[i], 0)
            mask = cv2.resize(original_mask, (256, 256))
            mask = mask/255
            mask = np.expand_dims(mask, axis = 2)
            yield (original_img, original_mask, img, mask)
def val_data_generator():
    test_images = os.listdir(test_images_path)
    test_masks = os.listdir(test_masks_path)
    while True:
        for i in range(len(test_images)):
            img = cv2.imread(test_images_path + "/" + test_images[i])
            img = cv2.resize(img, (256, 256))
            img = img/255
            mask = cv2.imread(test_masks_path + "/" + test_images[i], 0)
            mask = cv2.resize(mask, (256, 256))
            mask = mask/255
            mask = np.expand_dims(mask, axis = 2)
            yield (img, mask)

In [ ]:
unet_inputs = tf.keras.layers.Input(shape=(256, 256, 3))
######################################################################################################################################################
Encoder_1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(unet_inputs)
Encoder_1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Encoder_1)
P1 = MaxPooling2D(pool_size=(2, 2))(Encoder_1)
########################################################################################################################
Encoder_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(P1)
Encoder_2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Encoder_2)
P2 = MaxPooling2D(pool_size=(2, 2))(Encoder_2)
##########################################################################################
Encoder_3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(P2)
Encoder_3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Encoder_3)
P3 = MaxPooling2D(pool_size=(2, 2))(Encoder_3)
############################################################
Encoder_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(P3)
Encoder_4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Encoder_4)
Dropout_layer_1 = Dropout(0.1)(Encoder_4)
P4 = MaxPooling2D(pool_size=(2, 2))(Dropout_layer_1)
##############################
Encoder_5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(P4)
Encoder_5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Encoder_5)
Dropout_layer_2 = Dropout(0.1)(Encoder_5)
##############################
up_sample_6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(Dropout_layer_2))
Conc_6 = concatenate([Dropout_layer_1,up_sample_6], axis = 3)
Decoder_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Conc_6)
Decoder_1 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Decoder_1)
############################################################
up_sample_7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(Decoder_1))
Conc_7 = concatenate([Encoder_3,up_sample_7], axis = 3)
Decoder_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Conc_7)
Decoder_2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Decoder_2)
##########################################################################################
up_sample_8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(Decoder_2))
Conc_8 = concatenate([Encoder_2,up_sample_8], axis = 3)
Decoder_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Conc_8)
Decoder_3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Decoder_3)
########################################################################################################################
up_sample_9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(Decoder_3))
Conc_9 = concatenate([Encoder_1,up_sample_9], axis = 3)
Decoder_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Conc_9)
Decoder_4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Decoder_4)
######################################################################################################################################################
Decoder_4 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(Decoder_4)
Unet_outputs = Conv2D(1, 1, activation = 'sigmoid')(Decoder_4)

model = tf.keras.Model(unet_inputs, Unet_outputs)
model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
model_checkpoint = ModelCheckpoint('polyp_UNET.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [ ]:
def batch_generator(batch_size, gen_x): 
    batch_features = [] 
    batch_labels = []
    for i in range(batch_size):
        x_train, y_train = next(gen_x)
        batch_features.append(x_train)
        batch_labels.append(y_train)
    batch_features = np.array(batch_features)
    batch_labels = np.array(batch_labels)
    batch_labels = tf.cast(batch_labels, tf.float32)
    return batch_features, batch_labels
x_train, y_train = batch_generator(300, train_data_generator())
history = model.fit(
        x_train, y_train,
        epochs=10, 
        steps_per_epoch=200, 
        validation_data=batch_generator(300, val_data_generator()), 
        validation_freq=1, 
        callbacks=[model_checkpoint],
    )

# Predictions

In [ ]:
model = load_model("polyp_UNET.hdf5")

In [ ]:
test_data = test_data_generator()

In [ ]:
validation_similarity_score=[]
for i in range(100):
    original_img, original_mask, X, y_true = next(test_data)
    y_pred = model.predict(np.expand_dims(X,0))
    _,y_pred_thr = cv2.threshold(y_pred[0,:,:,0]*255, 127, 255, cv2.THRESH_BINARY)
    y_pred = (y_pred_thr/255).astype(int)
    
    y_pred_original = cv2.resize(y_pred.astype(float), (original_img.shape[1],original_img.shape[0]), interpolation= cv2.INTER_LINEAR)
    # similarity_score = jaccard_score(original_mask[:,:,0]//255,y_pred_original.astype(int), average="micro")
    # validation_similarity_score.append(similarity_score)
    if i < 5:
        plt.figure(figsize=(20,10))
        plt.subplot(1,2,1)
        plt.imshow(original_img[...,::-1], 'gray', interpolation='none')
        plt.imshow(original_mask/255.0, 'jet', interpolation='none', alpha=0.4)
        plt.subplot(1,2,2)
        plt.imshow(original_img[...,::-1], 'gray', interpolation='none')
        plt.imshow(y_pred_original/255.0, 'jet', interpolation='none', alpha=0.4)
        plt.show()
# similarity_score_mean = validation_similarity_score.mean()
# print("Mean validation Jaccard (on original data):", similarity_score_mean)

